# **Cats and Dogs Classification with SVM**

**Step 1: Import Necessary Libraries**

In [ ]:
# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


**Step 2: Download and Prepare the Dataset**

In [ ]:
# Unzip the dataset
import zipfile
with zipfile.ZipFile("train.zip", "r") as zip_ref:
    zip_ref.extractall("data")


In [ ]:
# Define paths for cats and dogs images
cat_path = "data/train/cat"
dog_path = "data/train/dog"

# Display sample images
def display_sample_images(path, label):
    images = os.listdir(path)[:5]
    plt.figure(figsize=(15, 5))
    for i, img_name in enumerate(images):
        img = cv2.imread(os.path.join(path, img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(1, 5, i + 1)
        plt.imshow(img)
        plt.title(label)
        plt.axis("off")
    plt.show()

print("Sample Cat Images:")
display_sample_images(cat_path, "Cat")

print("Sample Dog Images:")
display_sample_images(dog_path, "Dog")


**Step 3: Preprocess the Dataset**

In [ ]:
# Parameters for preprocessing
IMG_SIZE = 64  # Resize images to 64x64

# Preprocess images
def preprocess_images(path, label):
    data = []
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize image
        img = img.flatten()  # Flatten the image into a 1D vector
        data.append((img, label))
    return data

# Load and preprocess images
cat_data = preprocess_images(cat_path, 0)  # Label for cats: 0
dog_data = preprocess_images(dog_path, 1)  # Label for dogs: 1

# Combine and shuffle the dataset
data = cat_data + dog_data
np.random.shuffle(data)


**Step 4:  Split Data into Training and Testing Sets**

In [ ]:
# Split data into features and labels
X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])

# Normalize the features (pixel values)
X = X / 255.0

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


**Step 5: Train the SVM Model**

In [ ]:
# Train an SVM classifier
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(X_train, y_train)

print("SVM model training complete.")


**Step 6: Evaluate the Model**

In [ ]:
# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))


**Step 7: Visualize Predictions**

In [ ]:
# Visualize some predictions
def visualize_predictions(X, y_true, y_pred, num_images=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        img = X[i].reshape(IMG_SIZE, IMG_SIZE, 3) * 255
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img.astype("uint8"))
        plt.title(f"True: {'Cat' if y_true[i] == 0 else 'Dog'}\nPred: {'Cat' if y_pred[i] == 0 else 'Dog'}")
        plt.axis("off")
    plt.show()

print("Sample Predictions:")
visualize_predictions(X_test[:5], y_test[:5], y_pred[:5])


**Step 8: Save Results**

In [ ]:
# Save the model
import joblib
joblib.dump(svm_model, "svm_dog_cat_classifier.pkl")

print("Model saved as 'svm_dog_cat_classifier.pkl'.")
